In [5]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, log_loss

In [6]:
def load_data(data_path: str) -> pd.DataFrame:
    df = pd.read_excel(data_path,
                       usecols=['발명의 명칭', '요약', '메인IPC2', '대표청구항'])
    df['input'] = df.apply(
        lambda row: f"{row['발명의 명칭']}\n\n{row['요약']}", axis=1)
    df = df.drop(columns=['발명의 명칭', '요약', '대표청구항'])
    df.columns = ['메인IPC2', '대표청구항']
    return df


# 훈련 데이터 로드
xlsx_path_train = 'dataset/DS학술제-모델링경진대회_Train.xlsx'
df_train = load_data(xlsx_path_train)

# 테스트 데이터 로드
xlsx_path_test = 'dataset/DS학술제-모델링경진대회_Valid.xlsx'
df_test = load_data(xlsx_path_test)

# Separate features and target
X_train = df_train['대표청구항']
y_train = df_train['메인IPC2']
X_test = df_test['대표청구항']
y_test = df_test['메인IPC2']

In [7]:
# 텍스트 데이터를 TF-IDF로 변환
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# SVM 모델 학습
classifier = SVC(kernel='linear', probability=True, random_state=42)
classifier.fit(X_train_tfidf, y_train)

SVC(kernel='linear', probability=True, random_state=42)

In [8]:
# 테스트 데이터로 예측
y_pred = classifier.predict(X_test_tfidf)
y_pred_proba = classifier.predict_proba(X_test_tfidf)

# 손실 계산
test_loss = log_loss(y_test, y_pred_proba)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)

# 분류 리포트 생성
report = classification_report(y_test, y_pred, digits=4)

# 결과 출력
print(f'테스트 손실 {test_loss:.4f}, 정확도 {accuracy:.4f}')
print('분류 리포트:')
print(report)

테스트 손실 0.4494, 정확도 0.8289
분류 리포트:
              precision    recall  f1-score   support

        G06F     0.7647    0.5253    0.6228        99
        G06Q     0.8398    0.9598    0.8958       448
        G16H     0.7727    0.3091    0.4416        55

    accuracy                         0.8289       602
   macro avg     0.7924    0.5981    0.6534       602
weighted avg     0.8214    0.8289    0.8094       602

